In [5]:
import pycutestmgr as cute
import time
import pdb 
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import scipy.sparse as spc
from ipsolver import parse_cutest_like_problem, ipsolver, parse_matlab_like_problem
%matplotlib inline
cute.clearCache('HS71')

# Import Problem

In [15]:
problems = [("CORKSCRW", {"T": 10}),  # 1
            ("CORKSCRW", {"T": 50}),  # 2
            ("COSHFUN", {"M": 20}),  # 3 --- Too many iterations !!!
            ("READING1", {"N": 100}), # 4 --- Fails !!!
            ("SVANBERG", {"N": 500}),  # 5
            ("ORTHREGF", {"NPTS": 20}),  # 6
            ("OPTMASS", {"N": 200}),  # 7 --- Fails !!!
            ("OPTCNTRL", {}), # 8 --- Fails !!!
            ]
number = 5
name = problems[number-1][0]
cute.clearCache(name)
cute.prepareProblem(name, sifParams=problems[number-1][1])
problem = cute.importProblem(name)
info = problem.getinfo()
x0 = info["x"]
n = info["n"]
m = info["m"]
print(n, m)
v0 = info["v"]
eq_list = info["equatn"]

def convert_to_inf(bounds):
    new_bounds = np.zeros_like(bounds)
    i = 0
    for b in bounds:
        if b >= 1e20:
            new_bounds[i] = np.inf
        elif b <= -1e20:
            new_bounds[i] = -np.inf
        else:
            new_bounds[i] = b
        i += 1
    return new_bounds

# Define upper and lower bound
c_lb = convert_to_inf(info["cl"])
c_ub = convert_to_inf(info["cu"])
lb = convert_to_inf(info["bl"])
ub = convert_to_inf(info["bu"])

# Define function and constraints
def fun(x):
    return problem.obj(x)[0]

def grad(x):
    _, g1 = problem.obj(x, True)
    return g1

def lagr_hess(x, v):
    def matvec(p):
        return problem.hprod(p, x, v)
    return spc.linalg.LinearOperator((n, n), matvec=matvec)

def lagr_hessm(x, v, vv):
    def matvec(p):
        return problem.hprod(p, x, v)
    return spc.linalg.LinearOperator((n, n), matvec=matvec)


def constr(x):
    return problem.cons(x)

def jac(x):
    _, A1 = problem.cons(x, True)
    return spc.csc_matrix(A1)

# Parse constraints
fun, grad, new_lagr_hess, n_total_ineq, constr_ineq, \
    jac_ineq, n_total_eq, constr_eq, jac_eq \
        = parse_cutest_like_problem(
            fun, grad, lagr_hess, n, constr, jac,
            c_lb, c_ub, lb, ub)


(500, 500)


# Optimize Problem

In [16]:
start_time = time.time()
x, info = ipsolver(fun, grad, new_lagr_hess, n_total_ineq, constr_ineq,
                jac_ineq, n_total_eq, constr_eq, jac_eq, x0)
total_time = time.time() - start_time

0 1.0 1.05198284238 1.0
1 7.0 1.13329133196 7.0
2 18.7015898525 -1.00689100352 49.0
3 9.31868727933 0.948418056981 9.31868727933
4 21.2675422703 -1.25416938385 65.2308109553
5 9.43475774712 0.717910556954 9.43475774712
6 18.8695154942 -0.206383124931 18.8695154942
7 9.43475774712 0.570733956722 9.43475774712
8 18.8695154942 0.00835095143745 18.8695154942
9 17.2428170953 0.0135143346646 18.8695154942
10 18.8695154942 0.508604079126 18.8695154942
11 11.9577152104 0.942273340876 37.7390309885
12 3.44070760507 0.832654978229 83.704006473
13 0.663610356479 0.904630371859 83.704006473
===> 0.0221043637263 0.0738233841971
0 12.8829468178 1.32973429392 418.520032365
1 9.30324579363 1.0742147315 418.520032365
2 0.881063114649 1.0038054968 418.520032365
===> 0.0177369407624 0.00680275643655
0 16.0464620305 1.31885516483 2092.60016182
1 2.31535210998 1.01063136526 2092.60016182
2 17.1857463588 0.253056119637 2092.60016182
3 0.262568112628 0.967942612562 2092.60016182
===> 0.00142496319065 0.00028

# Display Results

In [17]:
print('f = '+ str(info["fun"]))
print('optimality = ' + str(info["opt"]))
print('c violation = ' + str(info["constr_violation"]))
print('niter = ' + str(info["niter"]))
print('total time = ' + str(total_time))

f = 835.186934705
optimality = 7.79377620838e-09
c violation = 3.30660438036e-14
niter = 156
total time = 4.36801099777
